In [ ]:
!pip uninstall tensorflow --yes
!pip install keras
!pip install tensorflow==1.15.2
!pip install kashgari==1.1.5

***总结***

**1. 模型在标注地名与年号的任务上表现良好，但是抓取动词，官名的能力较差**


**2.由于算力原因我还没有有效地完成超参数的调试选择，这可以作为之后的一个实验方向**


我在这一notebook中基于Kashgari模块实现对中文的实词识别。

kashgari模块可以完成基于BERT的中文实词识别。该模块基于tensorflow1， 所以可能在硬件加速上存在一些问题。因此在colab工作环境下，需要先卸载原有的tensorflow2并重装为tensorflow1.

In [ ]:
import kashgari
import tensorflow as tf

In [ ]:
import json
import numpy as np
import pandas as pd
import time

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


进入本地工作环境

In [ ]:
file=open('/content/drive/My Drive/result.json')
ming=json.load(file)

result.json文件包含了明朝的训练数据，接下来将基于result.json划分训练集（train），验证集（validate）与测试集（test）

In [ ]:
#unpack .json file, wash the data and transfer them into ndarray
def dataGen(ming):
    person_ids=np.array(list(ming.keys()))
    person_ids.sort()

    #idNum=len(person_ids)

    x_data=[]
    y_data=[]

    indexer=0
    for person_id in person_ids:
        char_tag=ming[person_id]['char_tag']
        x_data.append([])
        y_data.append([])
        omit_len=len(person_id)
        for i in range(omit_len+1,len(char_tag)):
            x_data[indexer].append(char_tag[i][0])
            y_data[indexer].append(char_tag[i][1])
         
        indexer=indexer+1

    for i in range(0,len(y_data)):
      for j in range(0,len(y_data[i])):
        old_text=y_data[i][j]
        #convert labels like 'B_date_reign' into 'B-date-reign', which is readable by kashgari
        new_text=old_text.replace("_","-")
        y_data[i][j]=new_text


    return x_data,y_data,person_ids


#construct train,validate and test set
#train_set_rate indicates the proportion of trainning data
#validate_set_rate indicates the proportion of validation data
def splitTrain(x_data,y_data,person_ids,train_set_rate,validate_set_rate):
    x_data=np.array(x_data)
    y_data=np.array(y_data)
    
    temp=np.array([x_data,y_data])
    temp=temp.T
    
    data=pd.DataFrame(temp,index=person_ids,columns=['x_data','y_data'])
    np.random.seed(int(time.time()))
    data=data.sample(frac=1.0)
    
    idNum=len(person_ids)
    train_size=int(np.floor(idNum*train_set_rate))
    validate_size=int(np.floor(idNum*validate_set_rate))

    train_set=data[0:train_size]

    validate_set=data[train_size:train_size+validate_size]
    test_set=data[train_size+validate_size:idNum]
    return train_set,validate_set,test_set

x_data,y_data,person_ids=dataGen(ming)
t,v,testing=splitTrain(x_data,y_data,person_ids,0.7,0.2)

#generate x,y of train,validate and test
x_train=np.array(t['x_data'])
y_train=np.array(t['y_data'])
x_validate=np.array(v['x_data'])
y_validate=np.array(v['y_data'])

x_test=np.array(testing['x_data'])
y_test=np.array(testing['y_data'])

t.to_csv('train.csv')
v.to_csv('validate.csv')




tag_list = ['O',
        'B_date_reign', 'I_date_reign',
        'B_date_year', 'I_date_year',
        'B_office_voa', 'I_office_voa',
        'B_office_title', 'I_office_title',
        'B_place_placename', 'I_place_placename']

In [ ]:
#BiLSTM_CRF_Model is used
from kashgari.embeddings import BERTEmbedding
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.tasks.labeling import BiGRU_CRF_Model


选用的预训练模型是较小的预训练模型（'chinese_wwm_ex_L-12_H-768_A-12）。该模型为base模型(相比于其公布的large 模型）

预训练模型可以在以下网址下载：https://github.com/ymcui/Chinese-BERT-wwm

In [ ]:
#use chinese_wwm_ex_L-12_H-768_A-12 as the pretrained model
embed=BERTEmbedding('/content/drive/My Drive/chinese_wwm_ext_L-12_H-768_A-12',task=kashgari.LABELING,sequence_length=512)
print('success!')


/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


success!


In [ ]:
model=BiLSTM_CRF_Model(embed)

x_train=list(x_train)
y_train=list(y_train)
x_validate=list(x_validate)
y_validate=list(y_validate)

model.fit(x_train,y_train,x_validate=x_validate,y_validate=y_validate,epochs=2,batch_size=128)

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 512)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

KeyboardInterrupt: ignored

只跑两个epoch，因为多了实在短时间内跑不动。。。

In [ ]:
#import pandas as pd
model.save('/content/drive/My Drive/ming_ner4_bilstm.model')
#model.save


In [ ]:
from kashgari.utils import load_model
model1=load_model('/content/drive/My Drive/ming_ner4_bilstm.model')

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [ ]:
model1.evaluate(x_test,y_test)

           precision    recall  f1-score   support

      voa     0.6660    0.7244    0.6940      2315
    reign     0.9876    0.9512    0.9690      1086
placename     0.9005    0.9319    0.9159      1058
    title     0.7146    0.8025    0.7560      2521
     year     0.8605    0.9171    0.8879       760

micro avg     0.7708    0.8289    0.7988      7740
macro avg     0.7781    0.8289    0.8021      7740



'           precision    recall  f1-score   support\n\n      voa     0.6660    0.7244    0.6940      2315\n    reign     0.9876    0.9512    0.9690      1086\nplacename     0.9005    0.9319    0.9159      1058\n    title     0.7146    0.8025    0.7560      2521\n     year     0.8605    0.9171    0.8879       760\n\nmicro avg     0.7708    0.8289    0.7988      7740\nmacro avg     0.7781    0.8289    0.8021      7740\n'

从模型在测试集上的表现来看并不是非常突出，其对年号的识别能力相对较强，地名次之，但是在voa和title的标注上表现得较为一般。这可能归因于超参数的选择问题，因此该模型很有必要进行进一步的调参。

In [ ]:
#make prediction on jin data
import pandas as pd
jin=pd.read_table('/content/drive/My Drive/test_data.txt')
text=list(jin['content_without_name'])
#transfer the jin text into format that is readable by predict()
for i in range(0,len(text)):
  text[i]=list(text[i])

#jin_result=model1.predict(text)


在下面的文本中，我用训练的模型对前1500条传记进行了实词标注任务。（因为标注全部所需要的时间较多）

In [ ]:
#make prediction on jin, print(i) to see the process
jin=pd.read_table('/content/drive/My Drive/test_data.txt')
text=list(jin['content_without_name'])
for i in range(0,len(text)):
  text[i]=list(text[i])

pred=[]

for i in range(0,int(1500)):
  print(i)
  to_pred=[text[i]]
  pred.append(model1.predict(to_pred))

pred=pd.Series(pred)
pred.to_csv('/content/drive/My Drive/jin_pred.csv')
print('prediction accomplished')



In [ ]:
import pandas as pd
t=pd.Series(text)
t.to_csv('/content/drive/My Drive/text.csv')
t1=pd.read_csv('/content/drive/My Drive/text.csv')
t1

In [38]:
import pandas as pd
jin=pd.read_table('/content/drive/My Drive/test_data.txt')
jin_pred=pd.read_table('/content/drive/My Drive/jin_pred.csv')

在接下来的代码中，我将实词标注的结果与相应的传记进行了初步的比对.比对结果也在一定程度上符合测试集的结果，模型对年号和地名的预测能力相当不错，但是在动词，官名上表现较差。同时，出于不明原因，有部分汉字被直接标记为 I（而其前后均为标记O），这一bug会在接下来进行排查。

In [40]:
jin_pred=jin_pred[',0']
jin=jin['content_without_name']

import numpy as np

pred_text=[]

for i in range(0,len(jin_pred)):
  pred_text.append([])
  str1=''.join([str(i),jin[i]])
  str2=jin_pred[i]
  pred_text[i].append(str1)
  pred_text[i].append(str2)


In [65]:
pred_text=np.array(pred_text)
pred_text[3:10]

array([['3正大三年守魯山縣尉，修立臺碑。',
        '3,"[[\'B-date-reign\', \'I-date-reign\', \'B-date-year\', \'O\', \'B-office-voa\', \'B-place-placename\', \'I-place-placename\', \'O\', \'I-office-title\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\']]"'],
       ['4燕山人，明昌二年任萬泉縣令。',
        '4,"[[\'B-place-placename\', \'I-place-placename\', \'O\', \'O\', \'B-date-reign\', \'I-date-reign\', \'B-date-year\', \'O\', \'O\', \'B-office-title\', \'I-office-title\', \'I-office-title\', \'I-office-title\', \'O\']]"'],
       ['5大定三年任沛縣主簿兼縣尉。',
        '5,"[[\'B-date-reign\', \'I-date-reign\', \'B-date-year\', \'O\', \'B-office-voa\', \'B-place-placename\', \'I-office-title\', \'I-office-title\', \'I-office-title\', \'B-office-voa\', \'I-office-title\', \'I-office-title\', \'O\']]"'],
       ['6安邑人，進士。',
        '6,"[[\'B-place-placename\', \'I-place-placename\', \'O\', \'O\', \'O\', \'O\', \'O\']]"'],
       ['7濟寧人，約卒於明昌年間。',
        '7,"[[\'B-place-placename\', \'I-place-placename\', \'O\', \'O\', \'O\', \'O

上面的输出是对实词标记效果的初步展示。由于预测之后保存文件的格式问题，预测结果部份出现了较为累赘的字符串，我将在之后用正则表达式消除掉它们。

目前来看，使用BERT模型对金朝数据的初步标记已经完成，取得了一定的效果，但还有几个问题要在接下来解决：



**1.模型需要进一步选择超参数，并通过图标给出超参数选择过程中F1度量，recall等统计量的变化。这一过程有赖于之后的大量训练。**

**2.我还将把模型的标记结果在清洗后与之前LSTM的标记结果进行比对，以进一步研究BERT的效果**